In [1]:
# Import modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import itertools as it
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
from itertools import combinations
from statsmodels.stats.outliers_influence import variance_inflation_factor
import csv
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Import the raw UAV data from a CSV file
pathToFile = 'F:/Redaction/Bernadouze/Submitted/R2/Data_repository/Dataset_with_indices.csv'
rawData = pd.read_csv(pathToFile,sep=',')
UAVData = pd.DataFrame(rawData)
UAVData.head(40)

,Plot,Season,GEP (gCO2.m².hour-1),ER (gCO2.m².hour-1),NEE (gCO2.m².hour-1),B,G,R,RE,NIR,...,GLI,GNDVI,LCI,NDRE,NDVI,OSAVI,RCC,RVI,SIPI,TVI
0,P1,May,-1.86,1.30,-0.56,0.04,0.09,0.10,0.30,0.43,...,0.125000,0.653846,0.245283,0.178082,0.622642,0.478261,0.434783,4.300000,1.181818,1.059548
1,P3,May,-1.47,1.28,-0.19,0.04,0.09,0.07,0.25,0.40,...,0.241379,0.632653,0.319149,0.230769,0.702128,0.523810,0.350000,5.714286,1.090909,1.096416
2,P4,May,-1.25,0.89,-0.36,0.04,0.10,0.08,0.28,0.42,...,0.250000,0.615385,0.280000,0.200000,0.680000,0.515152,0.363636,5.250000,1.117647,1.086278
3,P5,May,-1.84,1.19,-0.65,0.04,0.07,0.05,0.18,0.34,...,0.217391,0.658537,0.410256,0.307692,0.743590,0.527273,0.312500,6.800000,1.034483,1.115164
4,P6,May,-1.27,0.96,-0.31,0.05,0.10,0.09,0.24,0.37,...,0.176471,0.574468,0.282609,0.213115,0.608696,0.451613,0.375000,4.111111,1.142857,1.052946
5,P7,May,-1.20,0.86,-0.34,0.03,0.06,0.04,0.18,0.32,...,0.263158,0.684211,0.388889,0.280000,0.777778,0.538462,0.307692,8.000000,1.035714,1.130388
6,P8,May,-0.93,0.97,0.04,0.04,0.06,0.07,0.18,0.26,...,0.043478,0.625000,0.242424,0.181818,0.575758,0.387755,0.411765,3.714286,1.157895,1.037187
7,P9,May,-1.12,0.94,-0.18,0.03,0.09,0.07,0.25,0.36,...,0.285714,0.600000,0.255814,0.180328,0.674419,0.491525,0.368421,5.142857,1.137931,1.083706
8,P1,June,-1.46,0.80,-0.66,0.02,0.06,0.05,0.22,0.35,...,0.263158,0.707317,0.325000,0.228070,0.750000,0.535714,0.384615,7.000000,1.100000,1.118034
9,P3,June,-1.23,0.76,-0.47,0.02,0.06,0.04,0.19,0.34,...,0.333333,0.700000,0.394737,0.283019,0.789474,0.555556,0.333333,8.500000,1.066667,1.135550


In [3]:
# Recsaling variables between -1 and 1 for feature importance analysis
scaler = preprocessing.StandardScaler()
temp = scaler.fit_transform(UAVData[['PAR','TEMP','DOY','B','G','R','RE','NIR','BCC','BRI','BWDRVI','CIgreen','CVI','EVI','EVI2','GARI','GCC','GLI','GNDVI','LCI','NDRE','NDVI','OSAVI','RCC','RVI','SIPI','TVI']])
dftemp = pd.DataFrame(temp, columns =['PAR','TEMP','DOY','B','G','R','RE','NIR','BCC','BRI','BWDRVI','CIgreen','CVI','EVI','EVI2','GARI','GCC','GLI','GNDVI','LCI','NDRE','NDVI','OSAVI','RCC','RVI','SIPI','TVI'])
UAVData[['PAR','TEMP','DOY','B','G','R','RE','NIR','BCC','BRI','BWDRVI','CIgreen','CVI','EVI','EVI2','GARI','GCC','GLI','GNDVI','LCI','NDRE','NDVI','OSAVI','RCC','RVI','SIPI','TVI']] = dftemp[['PAR','TEMP','DOY','B','G','R','RE','NIR','BCC','BRI','BWDRVI','CIgreen','CVI','EVI','EVI2','GARI','GCC','GLI','GNDVI','LCI','NDRE','NDVI','OSAVI','RCC','RVI','SIPI','TVI']]
UAVData.head(40)

,Plot,Season,GEP (gCO2.m².hour-1),ER (gCO2.m².hour-1),NEE (gCO2.m².hour-1),B,G,R,RE,NIR,...,GLI,GNDVI,LCI,NDRE,NDVI,OSAVI,RCC,RVI,SIPI,TVI
0,P1,May,-1.86,1.30,-0.56,0.898027,1.018077,1.938257,1.680524,1.026872,...,-1.053592,-0.369411,-1.757942,-1.723758,-1.453697,-0.550456,1.772118,-1.189586,1.940360,-1.461479
1,P3,May,-1.47,1.28,-0.19,0.898027,1.018077,0.587730,0.688739,0.606788,...,0.025701,-0.899549,-0.482891,-0.486361,-0.360996,0.093197,0.056132,-0.549435,0.146703,-0.345592
2,P4,May,-1.25,0.89,-0.36,0.898027,1.565104,1.037906,1.283810,0.886844,...,0.105649,-1.331513,-1.158668,-1.209001,-0.665187,-0.029150,0.332130,-0.759586,0.674250,-0.652433
3,P5,May,-1.84,1.19,-0.65,0.898027,-0.075976,-0.312622,-0.699759,-0.233380,...,-0.196762,-0.252082,1.089779,1.320237,0.208985,0.142136,-0.702861,-0.058006,-0.966601,0.221847
4,P6,May,-1.27,0.96,-0.31,1.975658,1.565104,1.488081,0.490382,0.186704,...,-0.576257,-2.355026,-1.113637,-0.900991,-1.645412,-0.927022,0.562128,-1.275084,1.171650,-1.661290
5,P7,May,-1.20,0.86,-0.34,-0.179605,-0.623002,-0.762798,-0.699759,-0.513436,...,0.227674,0.390143,0.720939,0.669862,0.678970,0.300247,-0.800168,0.485153,-0.942302,0.682656
6,P8,May,-0.93,0.97,0.04,0.898027,-0.623002,0.587730,-0.699759,-1.353604,...,-1.809618,-1.090988,-1.807289,-1.636015,-2.098213,-1.829405,1.306239,-1.454700,1.468345,-2.138264
7,P9,May,-1.12,0.94,-0.18,-0.179605,1.018077,0.587730,0.688739,0.046676,...,0.436861,-1.716354,-1.576160,-1.671016,-0.741914,-0.363013,0.428971,-0.808082,1.074457,-0.730282
8,P1,June,-1.46,0.80,-0.66,-1.257237,-0.623002,-0.312622,0.093669,-0.093352,...,0.227674,0.968146,-0.381891,-0.549751,0.297107,0.261425,0.756742,0.032520,0.326069,0.308726
9,P3,June,-1.23,0.76,-0.47,-1.257237,-0.623002,-0.762798,-0.501402,-0.233380,...,0.878476,0.785111,0.821885,0.740762,0.839754,0.541804,-0.281198,0.711469,-0.331605,0.838883


In [4]:
# Test models: each variable combinaison over 100 train/test splits
FeatureOfImportance = pd.DataFrame() # Put coef in a dataframe for feature importance analysis
c = 0

predVar = ['NEE (gCO2.m².hour-1)', 'ER (gCO2.m².hour-1)', 'GEP (gCO2.m².hour-1)']

for p in predVar: # Loop by variable to predict
    y = UAVData[p]
    X = UAVData[{'NDRE', 'BRI', 'NIR', 'BWDRVI', 'PAR', 'TEMP', 'CVI'}]
    for i in range(1,len(X.columns)+1): # Loop by the number of explanatory variables (X)
        SelectedFeatureNames = list(combinations(X.columns,i)) # List all combinations
        for elem in SelectedFeatureNames:
            for SplitNb in range(100): # Split the dataset in train and test dataset
                c = c+1
                train, test = train_test_split(UAVData, test_size=0.4)
                X_train = train.loc[:,elem]
                y_train = train.loc[:,p]
                X_test = test.loc[:,elem]
                y_test = test.loc[:,p]
                
                # Fit a model with train dataset
                model = sm.OLS(y_train,sm.add_constant(X_train,has_constant='add')).fit()
                
                # Make prediction on the test dataset
                ypred = model.predict(sm.add_constant(X_test,has_constant='add'))
                
                # Performance 
                AIC = model.aic
                R = np.corrcoef(y_test, ypred)
                RMSE = rmse(y_test, ypred)
                          
                # Store results for evauating feature importance
                
                Parameters = model.params[1:] # Store coefiscient without the constant
                Parameters['VarToPred'] = p
                Parameters['SplitId'] = SplitNb
                Parameters['NbOfExpVar'] = i
                Parameters['ExpVar'] = elem
                Parameters['R'] = abs(R[0,1])
                Parameters['RMSE'] = RMSE
                Parameters['AIC'] = AIC
                Parameters['r2'] = model.rsquared
                Parameters['Model'] = model
                
                FeatureOfImportance = FeatureOfImportance.append(Parameters,ignore_index=True)
                
# Save to dbf
FeatureOfImportance.to_csv('F:/Redaction/Bernadouze/Submitted/R2/Data_repository/ModelsUAV_FeatureImportance.csv',index=True)
FeatureOfImportance

,BWDRVI,VarToPred,SplitId,NbOfExpVar,ExpVar,R,RMSE,AIC,r2,Model,CVI,TEMP,NDRE,BRI,NIR,PAR
0,-0.046080,NEE (gCO2.m².hour-1),0.0,1.0,"(BWDRVI,)",0.579805,0.316063,14.030845,0.024472,<statsmodels.regression.linear_model.Regressio...,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.128698,NEE (gCO2.m².hour-1),1.0,1.0,"(BWDRVI,)",0.201000,0.245857,17.378596,0.133192,<statsmodels.regression.linear_model.Regressio...,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.128618,NEE (gCO2.m².hour-1),2.0,1.0,"(BWDRVI,)",0.301352,0.210867,19.388447,0.119652,<statsmodels.regression.linear_model.Regressio...,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.147541,NEE (gCO2.m².hour-1),3.0,1.0,"(BWDRVI,)",0.055126,0.248183,17.623448,0.178078,<statsmodels.regression.linear_model.Regressio...,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.065484,NEE (gCO2.m².hour-1),4.0,1.0,"(BWDRVI,)",0.270058,0.392144,3.480086,0.080201,<statsmodels.regression.linear_model.Regressio...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38095,-0.190979,GEP (gCO2.m².hour-1),95.0,7.0,"(BWDRVI, CVI, TEMP, NDRE, BRI, NIR, PAR)",0.395633,0.550515,29.600002,0.707498,<statsmodels.regression.linear_model.Regressio...,0.101047,-0.082754,-0.178427,-0.034460,0.003530,0.350136
38096,-0.005197,GEP (gCO2.m².hour-1),96.0,7.0,"(BWDRVI, CVI, TEMP, NDRE, BRI, NIR, PAR)",0.394860,1.833330,28.900510,0.677384,<statsmodels.regression.linear_model.Regressio...,0.106627,-0.310907,-0.175428,1.195839,0.099817,0.429902
38097,-0.064780,GEP (gCO2.m².hour-1),97.0,7.0,"(BWDRVI, CVI, TEMP, NDRE, BRI, NIR, PAR)",0.763300,0.494251,41.663185,0.543100,<statsmodels.regression.linear_model.Regressio...,0.266428,-0.018988,-0.066192,-0.156183,-0.081937,0.462153
38098,0.016988,GEP (gCO2.m².hour-1),98.0,7.0,"(BWDRVI, CVI, TEMP, NDRE, BRI, NIR, PAR)",0.538502,0.576472,24.530605,0.714265,<statsmodels.regression.linear_model.Regressio...,0.068312,-0.217814,-0.183791,-0.212757,-0.298362,0.500972
